In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')
path = '/content/gdrive/My Drive/CUAI/NSMC/'

Mounted at /content/gdrive/


In [2]:
def load_data(filename):
    with open(filename, 'r', encoding='UTF8') as f:
        id, document, label = [list() for _ in range(3)]
        for line in f.read().split('\n'):
            try:
                id_, document_, label_ = line.split('\t')  # 데이터가 Tab으로 구분되어 있음.
            except:
                print(f"Err line: '{line}'")

            id.append(id_)
            document.append(document_)
            label.append(label_)

    return id[1:], document[1:], label[1:] # 열 이름 제거

train_id, train_document, train_label = load_data(path+'ratings_train.txt')
test_id, test_document, test_label = load_data(path+'ratings_test.txt')
train_label = list(map(int, train_label))
test_label = list(map(int, test_label))

Err line: ''
Err line: ''


In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 3.3 MB/s 
     |████████████████████████████████| 448 kB 36.6 MB/s 


In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595747 sha256=9103987adc51278316915486b1174a74579e9cc5a5daefb7a4a3eeeb04edbd85
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 880 kB 44.5 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 6.6 MB 36.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl 

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-x0ip31v_/kobert-tokenizer_3c10624eb5444eb8aca6f1baf168328c
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-x0ip31v_/kobert-tokenizer_3c10624eb5444eb8aca6f1baf168328c
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=6933106dfb6db9df5aa56e49436f7009fb48c50b6f55b98e0307abae3c97b283
  Stored in directory: /tmp/pip-ephem-wheel-cache-ngq14rjk/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [7]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
#device = torch.device("cuda:0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
dataset_train = nlp.data.TSVDataset(path+"ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(path+"ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [13]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)


In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2344 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7077355980873108 train acc 0.5625
epoch 1 batch id 201 loss 0.45488038659095764 train acc 0.5871424129353234
epoch 1 batch id 401 loss 0.40222129225730896 train acc 0.6874220698254364
epoch 1 batch id 601 loss 0.5054371356964111 train acc 0.736922836938436
epoch 1 batch id 801 loss 0.4631950855255127 train acc 0.763791354556804
epoch 1 batch id 1001 loss 0.36590713262557983 train acc 0.7809222027972028
epoch 1 batch id 1201 loss 0.4370436668395996 train acc 0.7932712323064113
epoch 1 batch id 1401 loss 0.386447936296463 train acc 0.8022060135617416
epoch 1 batch id 1601 loss 0.27560606598854065 train acc 0.8103236258588382
epoch 1 batch id 1801 loss 0.28510797023773193 train acc 0.8166470016657412
epoch 1 batch id 2001 loss 0.3437504470348358 train acc 0.8225730884557722
epoch 1 batch id 2201 loss 0.3105081021785736 train acc 0.8272589164016356
epoch 1 train acc 0.8306158454209329


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 3 batch id 1 loss 0.34059232473373413 train acc 0.875
epoch 3 batch id 201 loss 0.12013451009988785 train acc 0.9234297263681592
epoch 3 batch id 401 loss 0.14509275555610657 train acc 0.9251480673316709
epoch 3 batch id 601 loss 0.27611634135246277 train acc 0.9277506239600666
epoch 3 batch id 801 loss 0.2723178267478943 train acc 0.9291510611735331
epoch 3 batch id 1001 loss 0.2839120030403137 train acc 0.9314747752247752
epoch 3 batch id 1201 loss 0.07515525817871094 train acc 0.9340913821815154
epoch 3 batch id 1401 loss 0.14409416913986206 train acc 0.9357044075660242
epoch 3 batch id 1601 loss 0.2560769319534302 train acc 0.9371193785134291
epoch 3 batch id 1801 loss 0.11115580052137375 train acc 0.9386104941699056
epoch 3 batch id 2001 loss 0.11583962291479111 train acc 0.940436031984008
epoch 3 batch id 2201 loss 0.24980837106704712 train acc 0.9413192866878691
epoch 3 train acc 0.9422550483503982


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 3 test acc 0.896639226342711


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.43678274750709534 train acc 0.875
epoch 4 batch id 201 loss 0.060812026262283325 train acc 0.9537468905472637
epoch 4 batch id 401 loss 0.12413353472948074 train acc 0.9563980673316709
epoch 4 batch id 601 loss 0.20773400366306305 train acc 0.9578826955074875
epoch 4 batch id 801 loss 0.2022932767868042 train acc 0.9589575530586767
epoch 4 batch id 1001 loss 0.2257670909166336 train acc 0.960679945054945
epoch 4 batch id 1201 loss 0.035924721509218216 train acc 0.9621539342214821
epoch 4 batch id 1401 loss 0.04862051457166672 train acc 0.9630397930049964
epoch 4 batch id 1601 loss 0.08069884777069092 train acc 0.9638507183010618
